In [1]:
import numpy as np
import os as os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
sns.set_style("darkgrid")

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# load data
path = '/kaggle/input/calabi-yau-cicy-4-folds/'

conf = np.load('/kaggle/input/calabi-yau-cicy-4-folds/conf.npy')
hodge = np.load(os.path.join(path, 'hodge.npy'))
direct = np.load(os.path.join(path, 'direct.npy'))
conf.shape, hodge.shape, direct.shape

((921497, 16, 20), (921497, 4), (921497,))

In [3]:
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

In [4]:
import sys
sys.path.append('/kaggle/input/calabi-yau-cicy-4-folds')
from CICY4_functions import data_generator, batch_gd_scheduler,  calc_accuracy_mr, plot_losses

In [5]:
from sklearn.model_selection import train_test_split

def train_test(X, y):
    X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 101, shuffle = True)
    
    X_train = torch.from_numpy(X_train.astype(np.float32))
    #only need reshape if the y dimension is 1
    #y_train = torch.from_numpy(y_train.astype(np.float32).reshape(-1, 1))
    y_train = torch.from_numpy(y_train.astype(np.float32))

    X_test = torch.from_numpy(X_test.astype(np.float32))
    #y_test = torch.from_numpy(y_test.astype(np.float32).reshape(-1, 1))
    y_test = torch.from_numpy(y_test.astype(np.float32))                         
    
    print(f'X_train shape: {X_train.shape}, \n y_train shape:{y_train.shape},\
                 \n X_test shape: {X_test.shape}, \n y_test shape:{y_test.shape}')
    return X_train, X_test, y_train, y_test

In [6]:
X = conf
y = hodge

X_train, X_test, y_train, y_test = train_test(X, y)

train_gen = lambda: data_generator(X_train, y_train)
test_gen = lambda: data_generator(X_test, y_test)

X_train shape: torch.Size([737197, 16, 20]), 
 y_train shape:torch.Size([737197, 4]),                 
 X_test shape: torch.Size([184300, 16, 20]), 
 y_test shape:torch.Size([184300, 4])


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# CNN-RNN hybrid

In [8]:
##################### CNN ###############################
class CNN_block(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,128, 4, 1)
        #self.bn1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128,64, 3, 1)
        #self.bn2 = nn.BatchNorm2d(64)
        self.mxpool = nn.MaxPool2d(2,2)
        self.flat = nn.Flatten()
        self.conv_total = nn.Sequential(
            self.conv1,
            #self.bn1,
            self.mxpool,
            #self.bn2,
            self.conv2,
            self.mxpool,
        )

    def forward(self,x):
        x = F.relu(self.conv_total(x))
        #reshape is the same as flat(x)
        #x = x.reshape(x.shape[0], -1)
        x = self.flat(x)
        
        return x

In [9]:
cnn_block = CNN_block()
cnn_block.to(device)

CNN_block(
  (conv1): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (mxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (conv_total): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [10]:
class RNN_block(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_rnnlayers, n_outputs):
        super(RNN_block,self).__init__()
        self.D = n_inputs
        self.M = n_hidden
        self.K = n_outputs
        self.L = n_rnnlayers        
        #self.lstm = nn.LSTM(input_size = self.D,
        #                   hidden_size = self.M,
        #                   num_layers = self.L,
        #                   batch_first = True)    
        self.gru = nn.GRU(input_size = self.D,
                           hidden_size = self.M,
                           num_layers = self.L,
                           batch_first = True)
        #self.fc1 = nn.Linear(self.M, 128)
        #self.fc2 = nn.Linear(128, self.K)
    def forward(self, X):
        #input X is NxTxD
        #initial hidden states
        h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
        #c0 = torch.zeros(self.L, X.size(0), self.M).to(device)
        #get LSTM unit output:
        #output is NxTxM
        #out, _ = self.lstm(X, (h0,c0))
        out, _ = self.gru(X, h0)   
        #we only want h(T) at the final time step
        # output is now of shape (N, M)
        out = out[:, -1, :]

        return out

In [11]:
#gru_block = RNN_block(20, 64, 6, 4)
gru_block = RNN_block(20, 100, 8, 4)
gru_block.to(device)

RNN_block(
  (gru): GRU(20, 100, num_layers=8, batch_first=True)
)

In [12]:
class CNN_RNN_hybrid(nn.Module):
    def __init__(self, cnn_block, rnn_block, feat_vec_size):
        super(CNN_RNN_hybrid, self).__init__()
        self.cnn_block = cnn_block
        self.rnn_block = rnn_block
        self.feat_vec_size = feat_vec_size
        self.fc1 = nn.Linear(self.feat_vec_size, 512)
        self.fc2 = nn.Linear(512, 4)
        
    def forward(self, x):
        #output of cnn block is (N,384)
        x1 = x.view(-1,1, 16,20)
        x1 = self.cnn_block(x1)
        #output of rnn block is (N,M = 64)
        x2 = self.rnn_block(x)
        #concatenate the 2 outputs to produce a feat vec (N, M+384)
        xx = torch.cat([x1, x2], dim = 1)
        # pass through linear layers
        xx = self.fc1(xx)
        #final output is 4
        xx = self.fc2(xx)
        
        return xx

In [13]:
model = CNN_RNN_hybrid(cnn_block, gru_block, 100+384)
model.to(device)

CNN_RNN_hybrid(
  (cnn_block): CNN_block(
    (conv1): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (conv2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (mxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flat): Flatten(start_dim=1, end_dim=-1)
    (conv_total): Sequential(
      (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (rnn_block): RNN_block(
    (gru): GRU(20, 100, num_layers=8, batch_first=True)
  )
  (fc1): Linear(in_features=484, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
)

In [14]:
#count the number of parameters in the model
params = [p.numel() for p in model.parameters() if p.requires_grad]
print(sum(params))

787140


# Shape Tracing

In [15]:
X = X_train[0].to(device)
y = y_train[0].to(device)
print('RNN BLOCK SHAPE TRACING')
print(f'Original shape of the image before passing through the network: \n {X.shape}\n')

print('Reshape the size to take in account the batch number')
X = X.view(1,16,20)
print(f'The new size is {X.shape}\n')

h0 = torch.zeros(gru_block.L, X.size(0), gru_block.M).to(device)
#c0 = torch.zeros(gru_block.L, X.size(0), gru_block.M).to(device)
X, _ = gru_block.gru(X, h0)
print(f'Shape of the image after passing through the {gru_block.gru}: \n {X.shape}\n')

#get only the h(T) at the last time step
Xg = X[:, -1, :]
print(f'The final output shape is {Xg.shape}')
#print(X)
#print(f'Target: {y}')

RNN BLOCK SHAPE TRACING
Original shape of the image before passing through the network: 
 torch.Size([16, 20])

Reshape the size to take in account the batch number
The new size is torch.Size([1, 16, 20])

Shape of the image after passing through the GRU(20, 100, num_layers=8, batch_first=True): 
 torch.Size([1, 16, 100])

The final output shape is torch.Size([1, 100])


In [16]:
X = X_train[0].to(device)
print('CNN BLOCK SHAPE TRACING')
print(f'Original shape of the image before passing through the network: \n {X.shape}\n')

print('Reshape the size to take in account the batch number')
X = X.view(1,1,16,20)
print(f'The new size is {X.shape}\n')

X = cnn_block.conv_total(X)
print(f'Original shape of the image after passing through {cnn_block.conv_total}: \n {X.shape}\n')

#X = X.reshape(X.shape[0], -1)
#print(f'After reshaping: {X.shape}') [1,384]
Xc = cnn_block.flat(X)
print(f'Original shape of the image after passing through {cnn_block.flat}: \n {Xc.shape}\n')

CNN BLOCK SHAPE TRACING
Original shape of the image before passing through the network: 
 torch.Size([16, 20])

Reshape the size to take in account the batch number
The new size is torch.Size([1, 1, 16, 20])

Original shape of the image after passing through Sequential(
  (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
): 
 torch.Size([1, 64, 2, 3])

Original shape of the image after passing through Flatten(start_dim=1, end_dim=-1): 
 torch.Size([1, 384])



In [17]:
X_cat = torch.cat([Xc, Xg], dim=1) 
Xg.shape, Xc.shape, X_cat.shape

(torch.Size([1, 100]), torch.Size([1, 384]), torch.Size([1, 484]))

In [18]:
X = X_train[0].to(device)
print('CNN-RNN HYBRID BLOCK SHAPE TRACING')
print(f'Original shape of the image before passing through the network: \n {X.shape}\n')

print('Reshape the size to take in account the batch number')
X = X.view(1,16,20)
print(f'The new size is {X.shape}\n')

X = model(X)
print(f'Shape  after passing through the entire network: \n {X.shape}\n')

CNN-RNN HYBRID BLOCK SHAPE TRACING
Original shape of the image before passing through the network: 
 torch.Size([16, 20])

Reshape the size to take in account the batch number
The new size is torch.Size([1, 16, 20])

Shape  after passing through the entire network: 
 torch.Size([1, 4])



# Train loop with scheduler

https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.CosineAnnealingLR.html

https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html

https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.OneCycleLR.html

https://residentmario.github.io/pytorch-training-performance-guide/lr-sched-and-optim.html



In [19]:
max_epoch = 200
criterion = nn.MSELoss()
batch_size = 128
import torch.optim.lr_scheduler as lr_scheduler
#optimizer = torch.optim.SGD(model.parameters())

#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')


optimizer = torch.optim.AdamW(model.parameters())
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
#scheduler = lr_scheduler.OneCycleLR(optimizer=optimizer, epochs=max_epoch,
#            pct_start=0.0, steps_per_epoch=len(X_train)//batch_size,
#            max_lr= 0.01, div_factor=25, final_div_factor=4.0e-01)

In [20]:
train_losses, test_losses=batch_gd_scheduler(model, criterion, optimizer, train_gen, test_gen, scheduler, 
                                             max_epoch, device,  cnn=False)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 1/200, train loss:  685.8872          test_loss:  504.9681, duration: 0:00:45.100572,           learning rate: (0.001, 0.0005050107412538795)
Epoch: 2/200, train loss:  450.3318          test_loss:  413.3514, duration: 0:00:44.657776,           learning rate: (0.0005050107412538795, 0.00025245496761324684)
Epoch: 3/200, train loss:  354.7666          test_loss:  328.7154, duration: 0:00:44.663661,           learning rate: (0.00025245496761324684, 4.0169584454489425e-05)
Epoch: 4/200, train loss:  316.7877          test_loss:  316.3518, duration: 0:00:44.578267,           learning rate: (4.0169584454489425e-05, 0.0007060114411411234)
Epoch: 5/200, train loss:  300.4137          test_loss:  272.4715, duration: 0:00:44.568447,           learning rate: (0.0007060114411411234, 0.00014329736426895739)
Epoch: 6/200, train loss:  256.7443          test_loss:  262.4693, duration: 0:00:44.634968,           learning rate: (0.00014329736426895739, 0.0008569508003124687)
Epoch: 7/200, train 

In [21]:
train_acc, test_acc = calc_accuracy_mr(model, train_gen , test_gen, device = device, cnn= False)
print(f'Train accuracy for h11:{train_acc[0]:.4f}, Test accuracy for h11: {test_acc[0]:.4f}')
print(f'Train accuracy for h21:{train_acc[1]:.4f}, Test accuracy for h21: {test_acc[1]:.4f}')
print(f'Train accuracy for h31:{train_acc[2]:.4f}, Test accuracy for h31: {test_acc[2]:.4f}')
print(f'Train accuracy for h22:{train_acc[3]:.4f}, Test accuracy for h22: {test_acc[3]:.4f}')

Train accuracy for h11:0.3838, Test accuracy for h11: 0.3852
Train accuracy for h21:0.5351, Test accuracy for h21: 0.5350
Train accuracy for h31:0.1309, Test accuracy for h31: 0.1267
Train accuracy for h22:0.0348, Test accuracy for h22: 0.0329


In [22]:
torch.save(model, '/kaggle/working/saved_models/CNN_GRU_hybrid_cicy4_Hodge_v1.pt')